# AI Game Playing Algorithms with Checkers

Ryan Williams, Ben Newell, Chris Haynes

## Introduction

## Adversarial Search

Checkers is a zero-sum game, meaning the payoff for each player at the end of the game is equal and opposite. For this reason, adversarial search algorithms like `minimax` and `negamax` can be used to play the game of checkers. Through a process similar to a depth first search, the algorithms search through each possible move from the initial player. Then, from each of these moves, the algorithm searches each move from there for the opposing player. Once the depth limit or leaf nodes are reached, the utility is returned and the player chooses the move that optimizes their utility. From there, in the next recursion up, that player chooses it's optimum move and so on. This process results in one of the key components of adversarial search algorithms, each player assumming the other is playing optimally. Both of the two adversarial search algorithms, `minimax` and `negamax` follow this same process. The way they differ is in how players' utility is determined, which will be explained below.

### Necessary funtions added to Board class

#### isOver

`isOver` determines if the game is over, and if it is, gives the winner. It does this by checking the valid moves for each player. If the player has no valid moves, they have lost and the other player is the winner. This can be done by taking all of the opposing pieces (no pieces mean no valid moves) or by trapping all of the opponents pieces so they cannot move. If both players have valid moves they can do, the game is not over yet so the function returns `False` for the result and `None` for the winner.

isOver(self)

Parameters:
* None

Return:
* boolean - true if game is over, false otherwise
* winner - RED or BLACK depending on the winner. If game is not over, value is None

#### getUtility

`getUtility` gets the utility for the calling player at the specific point in the game. If type is `nm` and the game is over, the function will return 2 if the calling player won and -2 if the calling player lost. If type is `mm`, however, the function will return 2 or -2 if the calling player won, 2 if it's the maximizer, -2 if it's the minimizer. If the calling player lost, -2 is returned if the caller is the maximizer and 2 is returned if the calling player is the minimizer.

If the game is not over, then a heuristic is used to determine utility. The number of red and black pieces are counted with kings counting as two. If the type is `nm`, Whichever color has more pieces is awarded a utility of 1 and the color with less pieces receives a utility of -1 . Of course, if the type is `mm`, then the utilities are slightly different, with 1 going to maximizer if it has more pieces and -1 going to minimizer if it has more pieces. Therefore, the maximizer will get a utility of -1 if it has less pieces and the minimizer will get 1 if it has less. Finally, if the number of pieces is equal, a utility of 0 is returned.

The utility function was made in this way so that winning supercedes all. At first, the thought was to use 1 as the utility for both winning and having more pieces, but we decided that it makes the most sense for the algorithm to choose the winning move if there is an option between that and merely having more pieces. The most simple way to do this was to set winning utility as greater than the heuristic so it would always be chosen.

getUtility(self, type='nm', maximizePlayer=Color.RED)

Parameters:
* type: 'mm' or 'nm' depending on whether the algorithm is `minimax` or `negamax` since they give different utilities to each player
* maximizePlayer: Color of the maximizing player

Return:
* integer - represents the utility given at this stage of the game

### Minimax

#### Algorithm

The two players in the minimax algorithm are `max` and `min`. `Max` will look to maximize the utility and `min` will look to minimize the utility. For our case, in our checkers implementation, a win for `max`is worth 2 and a win for `min` is worth -2. `Minimax` searches through every possible move in the game until the depth limit, assuming on the way that the opponent will play optimally. If the depth limit is reached, a simple heuristic is used to see who has the advantage. All of the pieces for each player will be counted, with kings counting as two. If `max` has more, a utility of 1 will be awarded; if `min` has more, however, a utility of -1 is awarded. 

One thing added to this algorithm to cut down run time was a `break` statement used if the returned utility is 2 for the maximizer or -2 for minimizer. This was done because these values signify a win for their respective player so there is no reason to search any other nodes on that level of the tree. Even though a win could be achieved with a different move yet to be searched from that state, the algorithm wouldn't even choose that path because the returned value would equal the `bestValue` but not be greater than it, so `bestValue` would not be updated anyway.

#### Alpha beta pruning

Alpha beta pruning is a technique used in adversarial search algorithms to limit the number subtrees that need to be searched. It does this by taking the best value found so far for one player, represented by the variable `alpha`, and comparing it to the best value found so far for the other player, represented by the variable `beta`. If `alpha` is ever greater than `beta`, all other subtrees at that level do not need to be searched and can be pruned because, even if they were to be searched, they would not be chosen by the algorithm. To explain further, even though the current node may have chosen a value in the pruned subtree, the node above the current is guarenteed to not choose anything in that subtree since it has already found a better value in a previous subtree. As a result, cutting off these subrees saves a lot of time.

The implementation of alpha beta pruning is simple for `minimax`. First, The parameters `alpha` and `beta` are added to the funtion with `alpha` always representing the best value for the maximizer and `beta` always representing the best value for the minimizer. Inside the function, if the current player is the maximizer, after a value and a move is returned by the recursive call to `minimax`, the current best value found is compared to `alpha`. If `bestValue` is greater than `alpha`, `alpha` will be set to `bestValue`. If the current player is the minimzer, however, the current best value is compared to `beta` and if `bestValue` is less than `beta`, `beta` is set to `bestValue`. Finally, following this process for both the maximizer and minimizer, `alpha` is compared to `beta` and if `alpha` is greater than `beta`, a `break` statement is used and no more of the moves for that specific state are checked.

#### Function definition

minimax(board, depthLeft, alpha, beta, isMax=True)

Parameters:
* `board` : board object with necessary functions like validMoves, makeMove, etc.
* `depthLeft` : depth left before depth limit is reached and heuristic is used.
* `alpah` : alpha value for alpha beta pruning, represents best value so far for the maximizing player
* `beta` : beta value for alpha beta pruning, represents best value so far for the minimizing player
* `isMax` : deliniates whether this player is the maximizing player. 

Return:
* best value found and the resulting best move

### Negamax

#### Algorithm

As opposed to `minimax`, both players are looking to maximize their utility. As a result, a win for either player yields a utility of 2 and if the depth limit is reached, the heuristic is used giving either player a utility of one if they have more pieces than the other player. The algorithm flips the sign of the returned value from the recursive call, allowing both sides to look to maximize utility. Like the `minimax` algorithm, `negamax` also assumes the opponent will play optimally. 

Similar to `minimax`, a `break` statement is used if the utility is ever two, representing a win. The logic behind this operation is the same as the logic in `minimax`. If a win is found there is no reason to search any other moves because nothing better will be found, and, based on the way the algorithm updates the `bestValue`, once a win is discovered, another win will not result in `bestValue` being update.

#### Alpha beta pruning

Alpha beta pruning works very similarly in `negamax` to the way it does in `minimax`. `alpha` and `beta` are still added to the function definition as parameters, but one difference is `alpha` is the best value so far for the current player and `beta` is the best value so far for the opposing player. Additionally, in the recursive call to the negamax function, `-beta` is passed in as the `alpha` parameter and `-alpha` is passed in as the `beta` parameter. This is done so, inside the recursive call, the `alpha` and `beta` values correctly represent the best so far values for their respective players. Despite the differences, the function of alpha beta pruning remains the same for `negamax` as it is for `minimax`. If `alpha` is ever greater than `beta`, all of the subtrees at that level can be pruned because they would never be chosen by the algorithm.

#### Function definition

negamax(board, depthLeft, alpha, beta)

Parameters:
* `board` : board object with necessary functions like validMoves, makeMove, etc.
* `depthLeft` : depth left before depth limit is reached and heuristic is used.
* `alpah` : alpha value for alpha beta pruning, represents best value so far for the maximizing player
* `beta` : beta value for alpha beta pruning, represents best value so far for the minimizing player

Return:
* best value found and the resulting best move

### Examples

#### Minimax

In [6]:
def minimaxExample1():
    # Setup
    board = Board()
    piece = Piece(Color.RED, [3, 2])
    piece2 = Piece(Color.BLACK, [2, 1])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 2] = piece
    draught[2, 1] = piece2
    board.setBoard(draught)
    board.printState()

    value, move = minimax(board, 9, -inf, inf)
    board.makeMove(move)
    print(value)
    board.printState()

In [ ]:
def minimaxExample2():
    board = Board()
    piece = Piece(Color.RED, [3, 4])
    piece4 = Piece(Color.RED, [3, 2])
    piece5 = Piece(Color.RED, [2, 5])
    piece2 = Piece(Color.BLACK, [2, 3])
    piece3 = Piece(Color.BLACK, [0, 3])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 4] = piece
    draught[3, 2] = piece4
    draught[2, 5] = piece5
    draught[2, 3] = piece2
    draught[0, 3] = piece3
    board.setBoard(draught)
    board.printState()

    isMax = True
    isOver, _ = board.isOver()
    while not isOver:
        value, move = minimax(board, 10, -inf, inf, isMax)
        print("move: ", move)
        if move is None:
            print('move is None. Stopping')
            break
        print("\nPlayer", board.turn, "to", move, "for value", value)
        board.makeMove(move)
        print(board)
        isMax = not isMax
        isOver, _ = board.isOver()

In [ ]:
def minimaxExample3():
    board = Board()
    board.printState()

    isMax = True
    isOver, _ = board.isOver()
    while not isOver:
        if board.turn == Color.BLACK:
            move = board.validMoves()[int(len(board.validMoves()) / 2)]
            print("\nPlayer", board.turn, "to", move)
            board.makeMove(move)
        else:
            value, move = minimax(board, 10, -inf, inf, isMax)
            print("move: ", move)
            if move is None:
                print('move is None. Stopping')
                break
            print("\nPlayer", board.turn, "to", move, "for value", value)
            board.makeMove(move)
        print(board)
        isMax = not isMax
        isOver, _ = board.isOver()

#### Negamax

In [4]:
def negamaxExample1():
    # Setup
    board = Board()
    piece = Piece(Color.RED, [3, 2])
    piece2 = Piece(Color.BLACK, [2, 1])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 2] = piece
    draught[2, 1] = piece2
    board.setBoard(draught)
    board.printState()

    value, move = negamax(board, 9, -inf, inf)
    board.makeMove(move)
    print(value)
    board.printState()

In [ ]:
def negamaxExample2():
    # Setup
    board = Board()
    piece = Piece(Color.RED, [3, 4])
    piece4 = Piece(Color.RED, [3, 2])
    piece5 = Piece(Color.RED, [2, 5])
    piece2 = Piece(Color.BLACK, [2, 3])
    piece3 = Piece(Color.BLACK, [0, 3])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 4] = piece
    draught[3, 2] = piece4
    draught[2, 5] = piece5
    draught[2, 3] = piece2
    draught[0, 3] = piece3
    board.setBoard(draught)
    board.printState()

    isOver, _ = board.isOver()
    while not isOver:
        value, move = negamax(board, 5, -inf, inf)
        print("move: ", move)
        if move is None:
            print('move is None. Stopping')
            break
        print("\nPlayer", board.turn, "to", move, "for value", value)
        board.makeMove(move)
        print(board)
        isOver, _ = board.isOver()

In [ ]:
def negamaxExample3():
    board = Board()
    board.printState()

    isOver, _ = board.isOver()
    while not isOver:
        if board.turn == Color.BLACK:
            move = board.validMoves()[int(len(board.validMoves()) / 2)]
            print("\nPlayer", board.turn, "to", move)
            board.makeMove(move)
        else:
            value, move = negamax(board, 10, -inf, inf)
            print("move: ", move)
            if move is None:
                print('move is None. Stopping')
                break
            print("\nPlayer", board.turn, "to", move, "for value", value)
            board.makeMove(move)
        print(board)
        isOver, _ = board.isOver()

### Observations for Adversarial Search

First of all, time is a serious issue for these adversarial search algorithms. Without alpha beta pruning they run for an incredibly long time in the order of hours. As a result, alpha beta pruning is essentially a requirement for these algorithms as it cuts down the runtime immensely. However, even with the addition of alpha beta pruning, time is still a big issue for these adversarial search algorithms. Running an entire game against an opponent who chooses random moves still takes around 30 minutes, since most moves for each player need to be searched and there are 10<sup>20</sup>. 

Setting a depth limit and heuristic is also important for this problem since it saves a lot of time. However, the use of the depth limit could potentially lead to a move that isn't optimal being chosen, but the heuristic does its best in the attempt to choose what it sees as the best move once the depth limit is reached. Additionally, the depth limit must be chosen wisely. A lower depth limit will result in faster run time but a higher depth limit allows the algorithm to search deeper into the game, therefore giving it a higher likelyhood of choosing a good move. 

This leads to an issue encountered in the application of adversarial search to checkers. If the depth limit is low, when it is reached not enough has happened yet in the game for the heuristic to decide who has an advantage, and a utility of 0 is returned since both sides have an even amount of pieces. The utility being 0 happens for much of the first half of the game before a player is deemed to have an advantage, and, while bad moves are still avoided, the algorithm stll chooses the first of potentially many moves with a 0 utility possibly missing an optimum move.

Another issue observed due to the heuristic was sometimes a player seemed to decide to capture another piece just to get the piece count to be equal or in their favor, even if that capture is not the best move to do. As described earlier, the first half of the game is spent with the utility being 0 since both sides have an equal piece count. One potential reason for this is a player choosing to capture a piece to achieve the 0 utility when it would have a -1 utility (1 if the player is the minimizer in minimax) by not capturing. When looking at the board at these times, the capture seems illogical, but is still chosen because of the piece count reason. This same phenomenon is observed closer to the end of the game when an advantage can be derived. Seemingly illogical captures still take place just so the player will still have a greater piece count. However, even though these moves seem illogical from the onset because they lead to immediate capture of the origional piece or merely result in a perceived disadvantage, maybe prioritzing capturing and piece count is a worthwile stategy to consider in a checkers game.

### Further Research and Improvements

There is some further research that could be done for these adversarial search algorithms, most of which relating to speed. One of the biggest things that could be improved is creating a better heuristic. With a improved heuristic that can determine advantage earlier in the game with more levels of utilities, a wider range of values will be found earlier on. This would inevetably result in more pruning of subtrees since there will be more than just the five utility values of -2, -1, 0, 1, and 2 to compare. Additionally, the heuristic could be improved allowing for a prioritization of certain types of moves. For example, capturing or kinging could be prioritized if the utility would otherwise be the same. Even not moving certain pieces could be prioritized like not moving pieces in the back row. Statistics keeping on certain moves could also be done in an attempt to remember what moves are good, in a manner similar to reinforcement learning. This way, with knowledge of the statistics, certain moves could be chosen over others if the utility is otherwise the same.

In [3]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Williams-Newell-Haynes-Final-Project.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Williams-Newell-Haynes-Final-Project.ipynb is 2330


## Overall Observations